<h1>
<center>
Module 6 - from bag to matrix
</center>
</h1>
<div class=h1_cell>
<p>
This week I want to head in a slightly different direction than the bag of words approach. I want you to consider (and program) a co-occurrence matrix (comat). The comat can lead toward some very cool NLP methods.
</div>

#Weird but do it

You need to run your notebook with a GPU to get more RAM. Go to Runtime and then Change runtime type. Choose GPU. Now hover over the RAM slider on top right. You should see you have 25GB possible.

<h2>
<center>
Here's the idea
</center>
</h2>
<div class=h1_cell>
<p>
With bag of words, we threw away context information. If I look up bag_of_words['fawn'], I can see how many times fawn appeared but I don't know anything about what context it appeared in. The context I will be interested in to start with are the two words on either side of fawn, i.e., one word on left and one word on right. If fawn is at beginning or end of sentence, then we will only count one word.
<p>
To get this new information about context, I will have to go through all the sentences again, and for each word I will update what word is on either side of it. Let me try a small example. Pretend we only have 3 sentences (omitting any wrangling).
<p>
<pre>
I love programming. I love Math. I tolerate Biology.
</pre>
<p>
Check out the comat I would build from these sentences.
<p>
<img src='https://cdn-images-1.medium.com/max/1600/1*1p0geczj9KbJvwYi25B2Jg.png'>
<p>
You should notice that the row names and column names are symmetric. They are made up of the unique words in all the sentences. You should also note that sentences matter. For instance, if you remove the periods, then you would get `Math` coming right before `I`. So should be an entry for `[Math, I]` of `1`. But it is zero. We do not cross sentence boundaries to check for co-occurrences.
<p>
Also note that we are looking at either side of a word for co-occurrences. Hence, `I` shows up with `love` twice and `love` shows up with `I` twice.
</div>

<h2>
There is one parameter
</h2>


The parameter for a comat is how far from the target word we check. In the example above, the parameter value is 1 - check 1 word on either side. But I could set it to a larger integer. If I make it big enough, I'll likely get most of the sentence as the context.
<p>
Reminder: parameters like this in data science are often called hyper-parameters. The "K" in KNN is a hyper-parameter.

To start, you can assume the window is a constant 1. Later, I'll ask you to add a parameter that allows you to choose the window size, i.e.,  look 2 words out or 3 words out, etc. The larger the parameter, the more context you are picking up.

But to keep it simple, we assume window size 1 for now.

## Bring in your library functions

In [0]:
#flush the old directory
!rm -r  'w20_ds_library'

rm: cannot remove 'w20_ds_library': No such file or directory


In [0]:
my_github_name = 'FutureDeus'  #replace with your account name

In [0]:
clone_url = f'https://github.com/{my_github_name}/w20_ds_library.git'

In [0]:
#get the latest.
!git clone $clone_url 


Cloning into 'w20_ds_library'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 87 (delta 54), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (87/87), done.


In [0]:
from w20_ds_library import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


<h2>Challenge 1: A warm-up problem</h2>

Jargon alert. I will use *co-occurence matrix*, *comat* and *matrix* interchangeably in this module.

I am going to eventually ask you to build your own co-occurence matrix  using the sentences from the gothic authors. This means you need an algorithm for sliding a window along a sentence. But before getting to that on the big table, it helped me debug my code by first looking at a small problem to make sure I got that right. I'll lead you through a warm-up exercise to help you debug your algorithm for moving your window as you move across a sentence.

  I am not going to worry about processing speed but instead use a dataframe to hold my matrix (probably the slowest option) because it is easier to visualize. When we move to big table, you will need a Python data structure to speed things up.

Let's use the 3 sentences at the top of the notebook for our warm-up.

In [0]:
sentences_3 = ['I love programming.', 'I love Math.', 'I tolerate Biology.']  #Note that at top none of these wrangled. Even period left in as word.

In [0]:
wrangled_3 = [get_clean_words(swords,s) for s in sentences_3]  #I am going to wrangle them.
wrangled_3

[['love', 'programming'], ['love', 'math'], ['tolerate', 'biology']]

Write a function that will find the unique words (as a list) from the list of sentence words. In essence, flatten the list above and remove duplicates.


In [0]:
def unique_words(sentences:list) -> list:
  assert isinstance(sentences, list), f'sentences must be a list but saw a {type(sentences)}'
  assert all([isinstance(inner, list) for inner in sentences]), f'expecting sentences to be a list of lists'

  #your code here
  return list({j for i in sentences for j in i})


In [0]:
unique_3 = sorted(unique_words(wrangled_3)) # ['biology', 'love', 'math', 'programming', 'tolerate']
unique_3

['biology', 'love', 'math', 'programming', 'tolerate']

Building the table is pretty easy.

In [0]:
df_3 = pd.DataFrame(data=0, index=unique_3, columns=unique_3, dtype=int, copy=False)

In [0]:
df_3.head()

,biology,love,math,programming,tolerate
biology,0,0,0,0,0
love,0,0,0,0,0
math,0,0,0,0,0
programming,0,0,0,0,0
tolerate,0,0,0,0,0


Ok, go to it. Write the code that will fill in the table using window size 1 for sentences in wrangled_3.
<pre>

                                  biology  love	math	programming	tolerate
biology	                         0	   0	      0	   0	          1
love	                            0	   0	      1	   1              0
math	                            0	   1      	0	   0	          0
programming	                     0	   1	      0	   0	          0
tolerate	                        1	   0	      0	   0	          0
</pre>

In [0]:
#your code here

for i in wrangled_3:
  i_length = len(i)
  if i_length <= 1:
    continue
  else:
    for j in range(i_length):
      if (j + 1) < i_length:
        df_3.loc[i[j], i[j+1]] += 1
      if (j - 1) >= 0:
        df_3.loc[i[j], i[j-1]] += 1

In [0]:
df_3.head()  #check against my answer above

,biology,love,math,programming,tolerate
biology,0,0,0,0,1
love,0,0,1,1,0
math,0,1,0,0,0
programming,0,1,0,0,0
tolerate,1,0,0,0,0


<h2>
Challenge 2
</h2>

First let's bring the gothic table in.


In [0]:
import pandas as pd

gothic_table = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQqRwyE0ceZREKqhuaOw8uQguTG6Alr5kocggvAnczrWaimXE8ncR--GC0o_PyVDlb-R6Z60v-XaWm9/pub?output=csv',
                          encoding='utf-8')

In [0]:
gothic_table.head()


,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [0]:
len(gothic_table)

19579

##Not doing prediction

We are going to return to looking at vectors and cosine similarity. So we don't need the author column - we are not doing prediction. Only care about the text column.

<h2>
Move data out of gothic_table
</h2>

We have to start getting serious about processing time this week. We will be building some big data structures. The first thing to note is that `gothic_table.iterrows()` is about as slow as you can get. To avoid using it more than once, let's pull the text from the rows, wrangle it, then store the wrangled sentence in a list. So we will have a list of lists.

I think this will pay off for us later. We now can get rid of `gothic_table` and iterate over a pyhon list (fast!). Go ahead and build `sentences`.


##Important Point!

I want all the clean words in a sentence including duplicates. Last week we removed duplicates. Don't do that here. Here are first 2 entries that I get:
<pre>
[['process',
  'however',
  'afforded',
  'means',
  'ascertaining',
  'dimensions',
  'dungeon',
  'might',
  'make',
  'circuit',
  'return',
  'point',
  'whence',
  'set',
  'without',
  'aware',
  'fact',
  'perfectly',
  'uniform',
  'seemed',
  'wall'],
 ['never', 'occurred', 'fumbling', 'might', 'mere', 'mistake']]
 </pre>

In [0]:
%%time

#your code here (17s for me)

sentences = [get_clean_words(swords,row['text']) for index, row in gothic_table.iterrows()]


CPU times: user 18 s, sys: 45.9 ms, total: 18 s
Wall time: 18 s


In [0]:
sentences[:2]  #check against above

[['process',
  'however',
  'afforded',
  'means',
  'ascertaining',
  'dimensions',
  'dungeon',
  'might',
  'make',
  'circuit',
  'return',
  'point',
  'whence',
  'set',
  'without',
  'aware',
  'fact',
  'perfectly',
  'uniform',
  'seemed',
  'wall'],
 ['never', 'occurred', 'fumbling', 'might', 'mere', 'mistake']]

## Let's get the unique words

We can use these as our row and column labels.

In [0]:
%time vocab = unique_words(sentences)  #Wall time: 65.7 ms

CPU times: user 40 ms, sys: 968 µs, total: 41 ms
Wall time: 40.1 ms


In [0]:
'''
['aaem',
 'ab',
 'aback',
 'abaft',
 'abandon',
 'abandoned',
 'abandoning',
 'abandonment',
 'abaout',
 'abased']
 '''

sorted_vocab = sorted(vocab)
sorted_vocab[:10]

['aaem',
 'ab',
 'aback',
 'abaft',
 'abandon',
 'abandoned',
 'abandoning',
 'abandonment',
 'abaout',
 'abased']

In [0]:
len(sorted_vocab)  #24944 - so our comat will be 24944x24944

24944

<h2>
Challenge 3: you in charge
</h2>

Normally I would guide you through main steps and have you fill in pieces. I am going to cut you loose for this assignment. You can solve the problem in any way you want as long as you match my results.

I hope you have inferred from the sample comat above that your comat will be N by N where `N = len(sorted_vocab)`. Roughly 25K by 25K. Yikes. Every unique word is a row/column label.


## Ok, but what exactly should you do

You need a Python representation of the dataframe df_3 we created above. But now instead of a 5x5, you need a 24944x24944. This is your comat table.

You will then need to fill in values in your comat. As you go through sentences, you will check the words on either side of a word and update counts accordingly. For this challenge, you only need to look one word on either side, i.e., use a window size of 1.

##I am ok with you experimenting

I want you to write your own versions of KNN, Naive Bayes, etc. I think it gives you a deeper understanding of the models. And we can more easily make changes to them. But when it comes to a comat, I am ok if you want to try to reduce its memory footprint. And by doing so, you will likely improve speed.

That said, I used raw Python data structures and that is what you see for my times.

## Your target

I have a problem showing you a 25Kx25K matrix. Too much to print! So I instead will try this. I converted the data structure I was using to a list of tuples, sorted on the word. Each tuple/pair has the word and a sum of the columns for that word in my comat. So if you look at the row 'abandoned', and sum its 24944 columns, you get 53.
<pre>
[('aaem', 1),
 ('ab', 2),
 ('aback', 4),
 ('abaft', 2),
 ('abandon', 20),
 ('abandoned', 53),
 ('abandoning', 6),
 ('abandonment', 9),
 ('abaout', 46),
 ('abased', 1),
 ('abasement', 2),
 ('abashed', 2),
 ('abashment', 2),
 ('abate', 3),
 ('abated', 2),
 ('abatement', 4),
 ('abating', 1),
 ('abbey', 7),
 ('abbeys', 1),
 ('abbreviation', 3)]
 </pre>
Make sure you match.

In [0]:
#your code here for building your matrix
matrix = pd.DataFrame(data=0, index=sorted_vocab, columns=sorted_vocab, dtype=int, copy=False)
for i in sentences:
  i_length = len(i) - 1
  for j in range(i_length + 1):
    if (j - 1) >= 0:
      for k in range(1):
        matrix.loc[i[j], i[j-(k+1)]] += 1
    elif j != 0:
      for k in range(j):
        matrix.loc[i[j], i[j-(k+1)]] += 1
    if (j + 1) <= i_length:
      for k in range(1):
        matrix.loc[i[j], i[j+(k+1)]] += 1
    elif j != i_length:
      for k in range(i_length-(j)):
        matrix.loc[i[j], i[j+(k+1)]] += 1

In [0]:
#your code here for generating list of word-sum pairs to see if you match mine above
matrix_list = [(i, matrix[i].sum()) for i in sorted_vocab]
print(matrix_list[:20])

[('aaem', 1), ('ab', 2), ('aback', 4), ('abaft', 2), ('abandon', 20), ('abandoned', 53), ('abandoning', 6), ('abandonment', 9), ('abaout', 46), ('abased', 1), ('abasement', 2), ('abashed', 2), ('abashment', 2), ('abate', 3), ('abated', 2), ('abatement', 4), ('abating', 1), ('abbey', 7), ('abbeys', 1), ('abbreviation', 3)]


<h2>Some tests for you to run</h2>

First, define a function `get_matrix_row` that will pull out a row given a comat (i.e., matrix) and a row word. I'm not showing you my function because it is particular to my implementation of comat. You should define a version that makes sense for your comat.

## Object instead?

I was tempted to define a class comat here and then add get_matrix_row as a method. In essence, hide my implementation behind a set of getters and setters. I decided not to, but I could have gone either way.

In [0]:
def get_matrix_row(comat, word:str) -> list:
  assert isinstance(word, str), f'word must be a str but saw a {type(word)}'
  
  #your code here
  return comat.loc[word].tolist()

In [0]:
monster_row = get_matrix_row(matrix, 'monster')
monster_row.count(0)  #24875  - pretty sparse

24875

## One other thing

Make sure your cosine_similarity function handles a vector of 0s. This situation will crop up this week. In particular, I found these words have all zeroes:
<pre>
bawling
enmeshed
funnin
miscalculated
mountebanks
pleases
rouge
</pre>

In [0]:
cosine_similarity([0,0,0],[1,1,1])  #0.0

0.0

In [0]:
cosine_similarity(monster_row, monster_row)  #just checking - should be 1

1.0

Let's check out some words for similarity.

In [0]:
frankenstein_row = get_matrix_row(matrix, 'frankenstein')
cosine_similarity(monster_row, frankenstein_row)  #0.07273929674533079

0.07273929674533079

In [0]:
cosine_similarity(get_matrix_row(matrix, 'laboratory'), monster_row)  #0.0

0.0

Some more tests.

In [0]:
red_row = get_matrix_row(matrix, 'red')
blue_row = get_matrix_row(matrix, 'blue')
cosine_similarity(red_row, blue_row)  #0.19115696577049454

0.19115696577049454

In [0]:
green_row = get_matrix_row(matrix, 'green')

cosine_similarity(red_row, green_row)  #0.05128239532643232

0.05128239532643232

In [0]:
mad_row = get_matrix_row(matrix, 'mad')
sad_row = get_matrix_row(matrix, 'sad')
cosine_similarity(mad_row, sad_row)  #0.027606198265256506

0.027606198265256506

<h2>
Challenge 5
</h2>

Instead of just blindly searching for words that are similar, let's use one of our ideas from a previous model. As part of doing KNN, we computed the k closest rows. We then voted but we are going to skip that part. I only want the first part: produce a list of sorted distances.
<p>
BTW: this is where things slowed down and I was getting 4 minutes on colab.
</div>

In [0]:
def word_distances(matrix, word:str) -> list:
    assert isinstance(word, str), f'word must be a str but saw a {type(word)}'
    
    #your code here
    word_vect = get_matrix_row(matrix, word)
    return sorted([(index, cosine_similarity(word_vect, row.tolist())) for index, row in matrix.drop(word).iterrows()], key=lambda x: x[1], reverse = True)

Here is a fast test to see if you are on track.

In [0]:
test_word = sorted_vocab[0]  #get the first word as test
test_word

'aaem'

You will need to figure out how to slice off the first 1000 rows of your matrix. It will depend on how you are representing the matrix.

In [0]:
slice_1000 = matrix[:1000]

In [0]:
%time aaem_distances = word_distances(slice_1000, 'aaem') #only use first 1000 rows - Wall time: 10 s

CPU times: user 14.9 s, sys: 92 ms, total: 15 s
Wall time: 15 s


In [0]:
aaem_distances[:10]  #just for debugging

[('alludes', 0.4472135954999579),
 ('account', 0.05670479771237427),
 ('air', 0.038180177416060626),
 ('among', 0.029235267310234306),
 ('ab', 0.0),
 ('aback', 0.0),
 ('abaft', 0.0),
 ('abandon', 0.0),
 ('abandoned', 0.0),
 ('abandoning', 0.0)]

<pre>
[('alludes', 0.4472135954999579),
 ('account', 0.05670479771237427),
 ('air', 0.038180177416060626),
 ('among', 0.029235267310234306),
 ('ab', 0.0),
 ('aback', 0.0),
 ('abaft', 0.0),
 ('abandon', 0.0),
 ('abandoned', 0.0),
 ('abandoning', 0.0)]
 </pre>

If you are matching my results, go for the big enchilada. Find the closest words to `monster` in the full matrix.

In [0]:
%time monster_distances = word_distances(matrix, 'monster') #on colab Wall time: 4min 51s

CPU times: user 6min 12s, sys: 2.22 s, total: 6min 14s
Wall time: 6min 14s


In [0]:
'''
[('uncomplainingly', 0.23145502494313785),
 ('decipherer', 0.2182178902359924),
 ('hernani', 0.2182178902359924),
 ('mimes', 0.2182178902359924),
 ('skulking', 0.2182178902359924),
 ('ever', 0.19551857514700038),
 ('thought', 0.190143522496192),
 ('sentience', 0.17251638983558856),
 ('never', 0.1697162329953409),
 ('indistinctly', 0.1690308509457033)]
'''
monster_distances[:10]

[('uncomplainingly', 0.23145502494313785),
 ('decipherer', 0.2182178902359924),
 ('hernani', 0.2182178902359924),
 ('mimes', 0.2182178902359924),
 ('skulking', 0.2182178902359924),
 ('ever', 0.19551857514700038),
 ('thought', 0.190143522496192),
 ('sentience', 0.17251638983558856),
 ('never', 0.1697162329953409),
 ('indistinctly', 0.1690308509457033)]

In [0]:
%time red_distances = word_distances(matrix, 'red')

CPU times: user 6min 11s, sys: 44.8 ms, total: 6min 11s
Wall time: 6min 11s


In [0]:
'''
[('cancelled', 0.36380343755449945),
 ('engulfs', 0.36380343755449945),
 ('ignominy', 0.36380343755449945),
 ('satiated', 0.3451342449813167),
 ('hearkening', 0.30012252399939043),
 ('snub', 0.30012252399939043),
 ('certificates', 0.2970442628930023),
 ('unto', 0.2684377460965796),
 ('downcast', 0.25928148942086576),
 ('accrued', 0.25724787771376323)]
'''
red_distances[:10]

[('cancelled', 0.36380343755449945),
 ('engulfs', 0.36380343755449945),
 ('ignominy', 0.36380343755449945),
 ('satiated', 0.3451342449813167),
 ('hearkening', 0.30012252399939043),
 ('snub', 0.30012252399939043),
 ('certificates', 0.2970442628930023),
 ('unto', 0.2684377460965796),
 ('downcast', 0.25928148942086576),
 ('accrued', 0.25724787771376323)]

In [0]:
%time happy_distances = word_distances(matrix, 'happy')  #Wall time: 4min 55s

CPU times: user 6min 8s, sys: 55.7 ms, total: 6min 8s
Wall time: 6min 8s


In [0]:
'''
[('would', 0.3152106507470724),
 ('shall', 0.30266871241659793),
 ('could', 0.2919346540577332),
 ('might', 0.27465783756205087),
 ('saw', 0.2740787935057763),
 ('heard', 0.27217080979442226),
 ('seen', 0.2693112605933227),
 ('opportunity', 0.2657880650250036),
 ('forget', 0.25513799879424426),
 ('yet', 0.25421102244707794)]
'''

happy_distances[:10]

[('would', 0.3152106507470724),
 ('shall', 0.30266871241659793),
 ('could', 0.2919346540577332),
 ('might', 0.27465783756205087),
 ('saw', 0.2740787935057763),
 ('heard', 0.27217080979442226),
 ('seen', 0.2693112605933227),
 ('opportunity', 0.2657880650250036),
 ('forget', 0.25513799879424426),
 ('yet', 0.25421102244707794)]

#Challenge 6

Finally, I am going to ask you to generalize your code to allow creating different comats based on windown size. Our current matrix was created using window size of 1. I'd like to create a comat with an arbitrary size window that is specificed in a parameter `window_size`. Please fill in the function below so that you can call it with different window sizes and create the corresponding comat.

Again, I'll give you a target that has words and their row sums so you can see if you are matching my results.

In [0]:
def build_matrix(vocab:list, sentences:list, window_size):
  assert isinstance(vocab, list), f'vocab must be a list but saw a {type(vocab)}'
  assert all([isinstance(inner, str) for inner in vocab]), f'expecting vocab to be a list of strings'
  assert isinstance(sentences, list), f'sentences must be a list but saw a {type(sentences)}'
  assert all([isinstance(inner, list) for inner in sentences]), f'expecting sentences to be a list of lists'
  assert isinstance(window_size, int), f'window_size must be an int but saw a {type(window_size)}'

  #your code here
  matrix = pd.DataFrame(data=0, index=vocab, columns=vocab, dtype=int, copy=False)
  for i in sentences:
    i_length = len(i) - 1
    for j in range(i_length + 1):
      if (j - window_size) >= 0:
        for k in range(window_size):
          matrix.loc[i[j], i[j-(k+1)]] += 1
      elif j != 0:
        for k in range(j):
          matrix.loc[i[j], i[j-(k+1)]] += 1
      if (j + window_size) <= i_length:
        for k in range(window_size):
          matrix.loc[i[j], i[j+(k+1)]] += 1
      elif j != i_length:
        for k in range(i_length-(j)):
          matrix.loc[i[j], i[j+(k+1)]] += 1
  return matrix


##Test

We built matrix by hand earlier. Check if your new function produces the same thing when using windown size 1. BTW: this is where you need the extra RAM!

In [0]:
matrix1 = build_matrix(sorted_vocab, sentences, 1)  #about 3 minutes

In [0]:
matrix.equals(matrix1) #True - same as one we built earlier

True

##Test with window size 4

In [0]:
matrix4 = build_matrix(sorted_vocab, sentences, 4)  #try window size 4

Go ahead and generate word-sum pairs from matrix4 and check your answer against mine below.
<pre>
target[:20]

[('aaem', 4),
 ('ab', 6),
 ('aback', 11),
 ('abaft', 5),
 ('abandon', 66),
 ('abandoned', 186),
 ('abandoning', 21),
 ('abandonment', 35),
 ('abaout', 162),
 ('abased', 4),
 ('abasement', 8),
 ('abashed', 5),
 ('abashment', 7),
 ('abate', 12),
 ('abated', 8),
 ('abatement', 14),
 ('abating', 4),
 ('abbey', 25),
 ('abbeys', 4),
 ('abbreviation', 12)]
 </pre>

In [0]:
target = [(i, matrix4[i].sum()) for i in sorted_vocab]

In [0]:
target[:20]

[('aaem', 4),
 ('ab', 6),
 ('aback', 11),
 ('abaft', 5),
 ('abandon', 66),
 ('abandoned', 186),
 ('abandoning', 21),
 ('abandonment', 35),
 ('abaout', 162),
 ('abased', 4),
 ('abasement', 8),
 ('abashed', 5),
 ('abashment', 7),
 ('abate', 12),
 ('abated', 8),
 ('abatement', 14),
 ('abating', 4),
 ('abbey', 25),
 ('abbeys', 4),
 ('abbreviation', 12)]

Let's check distances we got with window size 1 against window size 4. My prediction is that they will be stronger now that we are taking more context.

In [0]:

cosine_similarity(get_matrix_row(matrix4, 'frankenstein'), get_matrix_row(matrix4, 'monster'))  #was 0.07273929674533079 - now 0.07604031408080632

0.07604031408080632

In [0]:
cosine_similarity(get_matrix_row(matrix4, 'red'), get_matrix_row(matrix4, 'blue'))  #was #0.19115696577049454 - now 0.24893166777030223

0.24893166777030223

In [0]:
cosine_similarity(get_matrix_row(matrix4, 'laboratory'), get_matrix_row(matrix4, 'monster'))  #was 0.0 - now 0.05321257140117508

0.05321257140117508

Check distances from monster in matrix4.

In [0]:
%time monster_distances4 = word_distances(matrix4, 'monster') #on colab Wall time: 4min 51s

CPU times: user 6min 12s, sys: 1.95 s, total: 6min 14s
Wall time: 6min 14s


Old:
<pre>
monster_distances[:10]
[('uncomplainingly', 0.23145502494313785),
 ('decipherer', 0.2182178902359924),
 ('hernani', 0.2182178902359924),
 ('mimes', 0.2182178902359924),
 ('skulking', 0.2182178902359924),
 ('ever', 0.19551857514700038),
 ('thought', 0.190143522496192),
 ('sentience', 0.17251638983558856),
 ('never', 0.1697162329953409),
 ('indistinctly', 0.1690308509457033)]

</pre>

New:
<pre>
monster_distances4[:10]

[('us', 0.3268544199618383),
 ('yet', 0.3235339332908998),
 ('would', 0.32106693219271015),
 ('one', 0.3189100302265062),
 ('little', 0.3020545565519684),
 ('see', 0.3015976395691136),
 ('even', 0.3010299409924667),
 ('ever', 0.2988210673905941),
 ('could', 0.2983649085654773),
 ('must', 0.29647320153988244)]
 </pre>

 Interesting. I think the old one (window size 1) is a little better than new one with window size 4.

In [0]:

monster_distances4[:10]

[('us', 0.3268544199618383),
 ('yet', 0.3235339332908998),
 ('would', 0.32106693219271015),
 ('one', 0.3189100302265062),
 ('little', 0.3020545565519684),
 ('see', 0.3015976395691136),
 ('even', 0.3010299409924667),
 ('ever', 0.2988210673905941),
 ('could', 0.2983649085654773),
 ('must', 0.29647320153988244)]

<h2>
Closing notes
</h2>

  Step back and see what is going on here. With `word_distances`, we are taking a target word and looking at its row vector. What is the row vector? It is the count of other words in  proximity of that word in all the sentences. So it represents "the company the word keeps". We now ask what other words keep similar company. If 2 words appear a lot together, that will make them similar. But they can also be similar if they never appear together. Instead, they appear with the same company often. For instance, take "*Once in a blue moon.*" and "*A blue dog democrat.*" Even if *moon* and *dog* may never appear together, they will be related through the *blue* column. So moon and dog will have some similarity. Kind of interesting.
  <p>
  
If we have time in the quarter, I'd like to follow up on the next step we can take once we have the comax. The general idea is that the comax is a very sparse matrix, consisting of mostly 0 entries. And as you have seen, it takes time to search it. There are linear-algebra  techniques for reducing a sparse matrix into a more dense matrix and hence, making them more computationally tractable to work with. One in particular is Singular-Value Decomposition (or SVD): https://en.wikipedia.org/wiki/Singular-value_decomposition. This is used with the Glove algorithm to take a comat, exactly like the one you built, and reduce it: https://nlp.stanford.edu/projects/glove/. You end up with a matrix of the same number of rows (one for each word) but roughly 300 columns (instead of 25K).  It's like magic!
<p>
This type of reduction is called "word embedding". It is like the 300 columns somehow capture the essence of the word in terms of its use in the context of a sentence. You can do some cool things with it. For instance, you can take the reduced vector for "king', subtract the reduced vector for 'male' and get the resulting vector for 'queen'. It is too wild.

I hope we can get to it toward the end of the quarter.
</div>